# Install Packages

In [1]:
!sudo apt install portaudio19-dev
!pip3 install -U scipy
!pip3 install wavio
!pip3 install sounddevice

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install


import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech()

import os
from google.colab import files

import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
import time

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/tortoise-tts
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post2-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0-py3-none-a

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:392: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

# EDIT output 'text' and 'preset' quality then RUN

In [6]:
# This is the text that will be spoken.
text = "Welcome to this presentation of Advanced Topics in Machine Learning. I am Alfio, or, at least, this is my generated voice. I hope you like it!"

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

# Sampling frequency
freq = 44100

# Recording duration
duration = 10

def record(freq, duration, output_file):

    print("-------------------")
    print("3...")
    time.sleep(1)
    print("2...")
    time.sleep(1)
    print("1...")
    time.sleep(1)
    print("Recording!")

    # Start recorder with the given values of duration and sample frequency
    recording = sd.rec(int(duration * freq),
                    samplerate=freq, channels=2)

    # Record audio for the given number of seconds
    sd.wait()

    # Convert the NumPy array to audio file
    wv.write(output_file, recording, freq, sampwidth=2)

In [4]:
recording_number = 1
CUSTOM_VOICE_NAME = "alfio"
if not os.path.exists(f"tortoise/voices/{CUSTOM_VOICE_NAME}"):
    os.makedirs(f"tortoise/voices/{CUSTOM_VOICE_NAME}")

for i in range(5):
    output_file = f"tortoise/voices/{CUSTOM_VOICE_NAME}/{CUSTOM_VOICE_NAME}_{recording_number}.wav"
    record(freq, 1, output_file)
    recording_number += 1

-------------------
3...
2...
1...
Recording!


PortAudioError: Error querying device -1

# Generate Audio File (clone voice) 👌

In [9]:
# Generate speech
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [02:04<00:00,  7.75s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 16/16 [00:30<00:00,  1.89s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [02:27<00:00,  2.72it/s]
